In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

# Load images
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize to fixed size
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            img_hog = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                           cells_per_block=(2, 2), block_norm='L2-Hys')
            images.append(img_hog)
            labels.append(label)
    return images, labels

# Load dataset
cat_dir = r'C:\Users\midhu\Downloads\test_set CAT DOG\test_set\cats'
dog_dir = r'C:\Users\midhu\Downloads\test_set CAT DOG\test_set\dogs'
cat_images, cat_labels = load_images_from_folder(cat_dir, 0)
dog_images, dog_labels = load_images_from_folder(dog_dir, 1)
X = np.array(cat_images + dog_images)
y = np.array(cat_labels + dog_labels)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define SVM model and parameters for Grid Search
svm_model = SVC()
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf']
}

# Use Grid Search to find the best parameters
grid_search = GridSearchCV(svm_model, param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predict and evaluate
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=["Cat", "Dog"]))



Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 71.60%
Classification Report:
               precision    recall  f1-score   support

         Cat       0.74      0.68      0.71       209
         Dog       0.69      0.75      0.72       196

    accuracy                           0.72       405
   macro avg       0.72      0.72      0.72       405
weighted avg       0.72      0.72      0.72       405

